In [1]:
import json, random, re, collections, itertools, base64, hashlib
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(x))
from pprint import pprint as pp
annot_path = Path("../annotations")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

## Read mapping

In [2]:
with (data_path/"annotaions.json").open('r') as f:
    annotations = json.load(f)

In [3]:
with (preprocessed_path/"resources.json").open('r') as f:
    resources = json.load(f)
with (preprocessed_path/"ingredients.json").open('r') as f:
    ingredients = json.load(f)
with (preprocessed_path/"labels.json").open('r') as f:
    idx2label = [tuple(t) for t in json.load(f)]
    label2idx={r:i for i,r in enumerate(idx2label)}

In [4]:
states = np.load(str(preprocessed_path/ "ultimate_state.npz" ))
states.files

['104600',
 '109875',
 '116231',
 '116803',
 '116687',
 '104786',
 '102138',
 '117061',
 '103308',
 '106850']

In [5]:
fname = states.files[0]
state = states[fname]
with (preprocessed_path / f"{fname}.json").open('r') as f:
    meta = json.load(f)
state.shape

(8, 17457)

In [6]:
annotation =  annotations["104600"][-1]
annotation

{'0': [['S1', 'Ieg3R-oQ_'], ['S1', 'LIMMEDIATE']],
 '1': [['S2', 'Ieg3R-oQ_'], ['S2', 'LIMMEDIATE']],
 '2': [['W1', 'I6xvdoZzb'],
  ['W1', 'IKeeuKzTy'],
  ['W1', 'IXclyBsV3'],
  ['W1', 'IpyJf3OVq'],
  ['W1', 'Iv00mZbtk'],
  ['W1', 'LIMMEDIATE'],
  ['W1', 'TBOWL']],
 '3': [['W1', 'I6xvdoZzb'],
  ['W1', 'IKeeuKzTy'],
  ['W1', 'IXclyBsV3'],
  ['W1', 'Ieg3R-oQ_'],
  ['W1', 'IpyJf3OVq'],
  ['W1', 'Iv00mZbtk'],
  ['W1', 'LIMMEDIATE'],
  ['W1', 'TBOWL']],
 '4': [['GW3', 'LIMMEDIATE'], ['GW3', 'MOIL'], ['GW3', 'TSKILLET']],
 '5': [['GW3', 'I6xvdoZzb'],
  ['GW3', 'IKeeuKzTy'],
  ['GW3', 'IXclyBsV3'],
  ['GW3', 'Ieg3R-oQ_'],
  ['GW3', 'IlGA3C7DK'],
  ['GW3', 'IpyJf3OVq'],
  ['GW3', 'Iv00mZbtk'],
  ['GW3', 'LIMMEDIATE'],
  ['GW3', 'MOIL'],
  ['GW3', 'TSKILLET']],
 '6': [['GW3', 'I6xvdoZzb'],
  ['GW3', 'IKeeuKzTy'],
  ['GW3', 'IXclyBsV3'],
  ['GW3', 'Ieg3R-oQ_'],
  ['GW3', 'IlGA3C7DK'],
  ['GW3', 'IpyJf3OVq'],
  ['GW3', 'Iv00mZbtk'],
  ['GW3', 'LCOLOR'],
  ['GW3', 'MOIL'],
  ['GW3', 'TLID'],
  ['G

In [13]:
max_ts = max(map(int, annotation.keys()))
new_state = None
state = {res: set() for res, desc in resources}
actions = []
for ts in range(1+max_ts):
    new_state = {res: set() for res, desc in resources}
    for resource, ing in annotation[str(ts)]: 
        new_state[resource].add(ing)
    for resource in new_state.keys():
        added_ings = new_state[resource]-state[resource]
        removed_ings = state[resource]-new_state[resource]
        for ing in added_ings:
            if ing.startswith("L"):
                if ing!="LIMMEDIATE":
                    actions.append((ts, "chef_check", ing, resource))
            elif ing.startswith("T"):
                actions.append((ts, "use", ing, resource))
            else:
                actions.append((ts, "add", ing, resource))
        for ing in removed_ings:
            if ing.startswith("I"):
                actions.append((ts, "remove", ing, resource))
    state = deepcopy(new_state)
pp (actions)

[(0, 'add', 'Ieg3R-oQ_', 'S1'),
 (1, 'remove', 'Ieg3R-oQ_', 'S1'),
 (1, 'add', 'Ieg3R-oQ_', 'S2'),
 (2, 'add', 'Iv00mZbtk', 'W1'),
 (2, 'add', 'I6xvdoZzb', 'W1'),
 (2, 'add', 'IpyJf3OVq', 'W1'),
 (2, 'add', 'IXclyBsV3', 'W1'),
 (2, 'use', 'TBOWL', 'W1'),
 (2, 'add', 'IKeeuKzTy', 'W1'),
 (2, 'remove', 'Ieg3R-oQ_', 'S2'),
 (3, 'add', 'Ieg3R-oQ_', 'W1'),
 (4, 'remove', 'Iv00mZbtk', 'W1'),
 (4, 'remove', 'I6xvdoZzb', 'W1'),
 (4, 'remove', 'IpyJf3OVq', 'W1'),
 (4, 'remove', 'IKeeuKzTy', 'W1'),
 (4, 'remove', 'IXclyBsV3', 'W1'),
 (4, 'remove', 'Ieg3R-oQ_', 'W1'),
 (4, 'add', 'MOIL', 'GW3'),
 (4, 'use', 'TSKILLET', 'GW3'),
 (5, 'add', 'Iv00mZbtk', 'GW3'),
 (5, 'add', 'I6xvdoZzb', 'GW3'),
 (5, 'add', 'IlGA3C7DK', 'GW3'),
 (5, 'add', 'IpyJf3OVq', 'GW3'),
 (5, 'add', 'IKeeuKzTy', 'GW3'),
 (5, 'add', 'IXclyBsV3', 'GW3'),
 (5, 'add', 'Ieg3R-oQ_', 'GW3'),
 (6, 'chef_check', 'LCOLOR', 'GW3'),
 (6, 'use', 'TLID', 'GW3'),
 (7, 'chef_check', 'LTIME', 'GW3')]
